#Mensurando e Classificando coerência em textos usando Llama v2.0 7B 8bit usando Langchain e Transformers by HuggingFace

Pré-requisitos:
- Lhama v2.0 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configure o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Referências**
https://medium.com/the-techlife/using-huggingface-openai-and-cohere-models-with-langchain-db57af14ac5b


**Notebook de referência:**

https://github.com/guardiaum/tutorial-sbbd2023/blob/main/Prompt_Engineering.ipynb


**Lista dos modelos:**

https://huggingface.co/models


**Artigos referências:**

https://dev.to/nithinibhandari1999/how-to-run-llama-2-on-your-local-computer-42g1


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 1 Preparação do ambiente
Preparação do ambiente para execução do exemplo.

##  1.1 Tempo inicial de processamento

In [ ]:
# Import das bibliotecas.
import time
from datetime import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

data_e_hora_atuais = datetime.now()
data_e_hora_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y %H:%M:%S')

print(data_e_hora_em_texto)

25/06/2024 19:32:39


## 1.2 Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Imprime linhas menores.

In [ ]:
def print_linhas_menores(texto, tamanho=120):
  for i in range(0, len(texto), tamanho):
    print(texto[i:i+tamanho])

## 1.3 Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## 1.4 Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch

def getDeviceGPU():
    """
      Retorna um dispositivo de GPU se disponível ou CPU.

      Retorno:
        `device` - Um device de GPU ou CPU.
    """

    # Se existe GPU disponível.
    if torch.cuda.is_available():

        # Diz ao PyTorch para usar GPU.
        device = torch.device("cuda")

        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")

    return device

In [ ]:
# Recupera o device com GPU ou CPU
device = getDeviceGPU()

GPU

In [ ]:
# Import de biblioteca
import torch

GPU_ENABLE = torch.cuda.is_available()

if GPU_ENABLE:
    print("GPU está disponível.")
else:
    print("GPU não está disponível.")

GPU está disponível.


Memória

Memória disponível no ambiente

In [ ]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

Versão Python

In [ ]:
# Biblioteca do sistema
import sys

print("Versão Python:", sys.version)

Versão Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


## 1.6 Instalação das bibliotecas

Instala langchain

In [ ]:
!pip install langchain==0.1.16

O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

In [ ]:
!pip install bitsandbytes==0.43.1

Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

https://pypi.org/project/accelerate/

In [ ]:
!pip install accelerate==0.29.3

A Biblioteca A Biblioteca Transformers fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração para Processamento de linguagem natural, Visão computacional, Áudio, etc.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
!pip install -U transformers==4.40.0

A Biblioteca huggingface-cli fornece vários comandos para interagir com o Hugging Face Hub a partir da linha de comando. Um desses comandos é o login, que permite aos usuários se autenticarem no Hub usando suas credenciais.

In [ ]:
#!pip install huggingface_hub==0.18.0

# 2 Parametrização

## Gerais

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf

In [ ]:
#nome_modelo = "meta-llama/Llama-2-7b-hf"
nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

#nome_modelo = "meta-llama/Llama-2-13b-hf"
# nome_modelo = "meta-llama/Llama-2-13b-chat-hf"

# Não roda pois exige GPU A100 e mais espaço em disco
#nome_modelo = "meta-llama/Llama-2-70b-hf"
# nome_modelo = "meta-llama/Llama-2-70b-chat-hf"

Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o **Access Token do HuggingFace** toda vez que for executar o notebook preencha a variável '<valor_do_acess_token>'.

Se for público crie a variável 'HF_TOKEN' com o valor do **Access Token do HuggingFace**. Abra o Google Colab e navegue até a nova seção 'Secrets' na barra lateral e adicione a variável.

In [ ]:
from huggingface_hub.hf_api import HfFolder

if IN_COLAB:
    from google.colab import userdata

    ACCESS_TOKEN  = userdata.get('HF_TOKEN')

    HfFolder.save_token(ACCESS_TOKEN)

else:
    ACESS_TOKEN = "<valor_do_acess_token"

    HfFolder.save_token(ACCESS_TOKEN)

Mostrando o usuário conectado

In [ ]:
# !huggingface-cli whoami

## Específicos

Parâmetros do modelo

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "COHQUAD_INIT_PTBR"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

# 3 - Carregando o LLM



## 3.1 - Carrega o tokenizador do LLM

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Tamanho do vocabulário

In [ ]:
print(len(tokenizer))

## 3.2 - Carregando o LLM

Carregando o **LLM** da comunidade HuggingFace.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

Carregamento LLama 2 com 4 bits

In [ ]:
# # Importando as bibliotecas do Modelo
# from transformers import BitsAndBytesConfig, AutoModelForCausalLM
# import torch
# import time

# # Guarda o tempo de início do carregamento do modelo
# tempo_inicio = time.time()

# # Carregando o Modelo da comunidade
# print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# # BitsAndBytes é um framework com funções customizadas para
# # otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
# quantization_config = BitsAndBytesConfig(
#    load_in_4bit=True, # Habilita a quantização de 4 bits para comprimir o modelo
#    bnb_4bit_quant_type="nf4", # Define o tipo de dados de quantização nas camadas (`fp4` e `nf4`).
#    bnb_4bit_use_double_quant=True, # Quantização aninhada, onde as constantes de quantização da primeira quantização são quantizadas novamente.
#    bnb_4bit_compute_dtype=torch.bfloat16 # # Os gradientes dos pesos são computados em 16-bit. Define o tipo computacional que pode ser diferente do tempo de entrada. Por exemplo, as entradas podem ser fp32, mas a computação pode ser definida como bf16 para acelerações.
# )

# # Se GPU Disponível
# if GPU_ENABLE:
#   # Carrega o modelo com a otimização BitsAndBytesConfig
#   print ("Carregando o LLM com GPU")
#   model = AutoModelForCausalLM.from_pretrained(nome_modelo,
#                                              #torch_dtype=torch.float16, #default
#                                              trust_remote_code=True, # Carrega de um repositório confiável
#                                              quantization_config=quantization_config,
#                                              device_map="auto"
#                                              )

# else:
#   # Carrega o modelo sem a otimização BitsAndBytesConfig
#   print ("Carregando o LLM sem GPU")
#   model = AutoModelForCausalLM.from_pretrained(nome_modelo,
#                                              #torch_dtype=torch.float16, #default
#                                              trust_remote_code=True, # Carrega de um repositório confiável
#                                              device_map="auto"
#                                              )

# # Coloca o modelo e modo avaliação
# model.eval()

# # Aumentar a velocidade
# # https://huggingface.co/docs/transformers/main/perf_torch_compile
# model = torch.compile(model)

# print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregamento LLama 2 com 8 bits

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# BitsAndBytes é um framework com funções customizadas para
# otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
quantization_config = BitsAndBytesConfig(
   load_in_8bit=True, # Habilita a quantização de 8 bits
)

# Se GPU Disponível
if GPU_ENABLE:
  # Carrega o modelo com a otimização BitsAndBytesConfig
  print ("Carregando o LLM com GPU")
  model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True, # Carrega de um repositório confiável
                                             quantization_config=quantization_config,
                                             device_map="auto"
                                             )
else:
  # Carrega o modelo sem a otimização BitsAndBytesConfig
  print ("Carregando o LLM sem GPU")
  model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True, # Carrega de um repositório confiável
                                             device_map="auto"
                                             )

# Coloca o modelo e modo avaliação
model.eval()

# Aumentar a velocidade
# https://huggingface.co/docs/transformers/main/perf_torch_compile
model = torch.compile(model)

print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

In [ ]:
print(model)

In [ ]:
print(model.config)

In [ ]:
print(model.config.max_position_embeddings)

Tamanho do vocabulário

In [ ]:
print(model.config.vocab_size)

## 3.3 - Configuração da geração de texto

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig

# Instância as configurações do modelo
generation_config = GenerationConfig.from_pretrained(nome_modelo)

print("GenerationConfig antes:\n",generation_config)

generation_config.max_new_tokens = 800 #Preenche até um comprimento máximo especificado com o argumento max_length ou até o comprimento de entrada máximo aceitável para o modelo se esse argumento não for fornecido.
generation_config.max_length = 800 # (Default 4096)
# Se do_sample é true setar temperature e top_p, caso contrário se do_sample é false remover temperature e top_p.
generation_config.do_sample = True # (Default True) Se definido como True, este parâmetro permite estratégias de decodificação como amostragem multinomial, amostragem multinomial de busca de feixe, amostragem Top-K e amostragem Top-p. Todas essas estratégias selecionam o próximo token da distribuição de probabilidade em todo o vocabulário com vários ajustes específicos da estratégia.
generation_config.temperature = 0.75 # (Default 0.6) A temperatura é um parâmetro que controla a aleatoriedade da saída do LLM. Uma temperatura mais alta resultará em um texto mais criativo e imaginativo, enquanto uma temperatura mais baixa resultará em um texto mais preciso e factual.
generation_config.top_k = 5  # Top-k diz ao modelo para escolher o próximo token entre os 'k' tokens principais de sua lista, classificados por probabilidade.
#generation_config.top_p = 0.1 # (Default 0.9) Top-p é mais dinâmico que top-k e é frequentemente usado para excluir resultados com probabilidades mais baixas. Portanto, se você definir p como 0,75, excluirá os 25% inferiores dos resultados menos prováveis.
#generation_config.repetition_penalty = 1.20 # Penaliza a repetição e visa evitar frases que se repetem sem nada de realmente interessante.
# generation_config.num_return_sequences=1, # Retorna uma única sentença da saída.

print("GenerationConfig depois:\n",generation_config)

## 3.4 - Cria o pipeline usando Langchain

Cria o pipeline com a classe [HuggingFacePipeline](https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html) do langchain.

Passagem direta do pipeline Huggingface.

Configura o pipeline do Huggingface usando o modelo e tokenizador previamente carregado e passa para o HuggingFacePipeline do langchain.

In [ ]:
# Import das bibliotecas
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Configura o pipeline do HuggingFace
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # (Default True) Langchain espera o texto completo
    generation_config=generation_config, # Passa as configurações da geração de texto para o pipeline
)

# Carrega o pipeline do Langchain
# https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
model_llm = HuggingFacePipeline(pipeline=pipe,
                                )

In [ ]:
print(model_llm)

# 4 - Classificando a coerência



## 4.1 - Prompt

```
Texto:"{TEXTO}"
Tarefa: Dado o texto acima forneça o resultado da classifação como coerente(1) ou incoerente(0) e justifique sua resposta no formato abaixo:
Resultado: <coerente(1)> ou <incoerente(0)>
Justificativa: <JUSTIFICATIVA>
```



In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def classificarCoerencia(texto):

  # Cria o texto de prompt
  prompt_template = """Texto:\"{TEXTO}\"
Tarefa: Dado o texto acima forneça o resultado da classifação como coerente(1) ou incoerente(0) e justifique sua resposta no formato abaixo:
Resultado: <coerente(1)> ou <incoerente(0)>
Justificativa: <JUSTIFICATIVA>
"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["TEXTO"],
                          template = prompt_template)

  #print("Prompt:\n", prompt.format(texto=texto))

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"TEXTO": texto})

  return resultado

Texto coerente

In [ ]:
texto = "Como empilhar um elemento em uma pilha?"

resultado = classificarCoerencia(texto)

print("Resposta: \n")
print_linhas_menores(resultado.get('text'))

Texto incoerente

In [ ]:
texto = "Como enfileirar um elemento em uma pilha?"

resultado = classificarCoerencia(texto)

#print("Resposta: \n" + resultado.get('text'))
print_linhas_menores(resultado.get('text'))

# 5 - Mensurando a coerência



## 5.1 - Prompt

```
Texto:"{TEXTO}"
Tarefa: Dado o texto acima, forneça uma pontuação de coerência do texto (5 - alta, 1 - baixa) e justifique sua resposta no formato abaixo:
Resultado: <PONTUAÇÃO>
Justificativa: <JUSTIFICATIVA>            
```



In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def mensurarCoerencia(texto):

  # Cria o texto de prompt
  prompt_template = """Texto:\"{TEXTO}\"
Tarefa: Dado o texto acima, forneça uma pontuação de coerência do texto (5 - alta, 1 - baixa) e justifique sua resposta no formato abaixo:
Resultado: <PONTUAÇÃO>
Justificativa: <JUSTIFICATIVA>
"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["TEXTO"],
                          template = prompt_template)

  #print("Prompt:\n", prompt.format(texto=texto))

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"TEXTO": texto})

  return resultado

Texto coerente

In [ ]:
texto = "Como empilhar um elemento em uma pilha?"

resultado = mensurarCoerencia(texto)

print("Resposta: \n")
print_linhas_menores(resultado.get('text'))

Texto incoerente

In [ ]:
texto = "Como enfileirar um elemento em uma pilha?"

resultado = mensurarCoerencia(texto)

#print("Resposta: \n" + resultado.get('text'))
print_linhas_menores(resultado.get('text'))